# **IMPLEMENTASI DAN EKSPERIMEN MODEL TRANSFORMER PADA TIGA DATASET UNTUK MENENTUKAN KINERJA TERBAIK**

Nama  : Maulana Seno Aji Yudhantara  
NRP   : 152022065  
Kelas : IFB-454 DEEP LEARNING  

## Konteks Proyek Kelompok:
Bagian dari tugas kelompok yang menguji tiga arsitektur deep learning berbeda pada tiga dataset yang sama. Notebook ini fokus pada eksperimen dan optimasi model Transformer.

## Tujuan Eksperimen:
Notebook ini bertujuan mengimplementasikan arsitektur Transformer dan melakukan eksperimen untuk mencari konfigurasi parameter terbaik pada tiga dataset yang telah ditentukan kelompok: BBC News, Sentiment140, dan FordA (UCR Time Series).  
Setiap dataset memiliki karakteristik berbeda (teks dan time series) sehingga diperlukan preprocessing dan penyesuaian model yang sesuai.

## Dataset yang Digunakan:
1. **BBC News**  
   - Jenis: Teks (klasifikasi topik berita)  
   - Deskripsi: Dataset berisi artikel berita BBC yang dikelompokkan ke dalam beberapa kategori topik.  

2. **Sentiment140**  
   - Jenis: Teks (klasifikasi sentimen tweet)  
   - Deskripsi: Dataset tweet Twitter yang dilabeli sentimen positif, negatif, dan netral.  

3. **FordA (UCR Time Series Classification)**  
   - Jenis: Time Series (klasifikasi deret waktu)  
   - Deskripsi: Dataset data sensor mobil Ford yang bertujuan klasifikasi kondisi berdasarkan sinyal time series.  

## Rencana Implementasi:
- Import library yang diperlukan  
- Memuat dan eksplorasi dataset  
- Preprocessing data sesuai kebutuhan Transformer  
- Definisi dan pelatihan model Transformer dengan variasi parameter  
- Evaluasi dan visualisasi hasil performa model  

## Output yang Dihasilkan:
- Source code lengkap model Transformer  
- Hasil evaluasi performa dan parameter terbaik  
- Penjelasan teknis dan hasil eksperimen sebagai bahan laporan kelompok  


# **Tahap 1 – Import Semua Library** 
Melakukan import seluruh library yang dibutuhkan mulai dari data handling, preprocessing, pembuatan model, hingga evaluasi dan visualisasi. Semua import dilakukan di satu cell agar terorganisasi rapi.

In [1]:
# Cell Code: Import Library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Untuk pemrosesan teks
import re
import string

# Library untuk machine learning dan deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel  # Jika pakai Huggingface Transformer (opsional)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Untuk time series (jika perlu preprocessing khusus)
from scipy import stats

# Untuk progress bar
from tqdm.notebook import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


ModuleNotFoundError: No module named 'transformers'

# **Tahap 2 – Load Dataset**
Memuat ketiga dataset ke memori notebook. Setiap dataset akan dibaca sesuai format dan disiapkan untuk tahap preprocessing.

- **Tahap 2.1:** Load dataset BBC News  
- **Tahap 2.2:** Load dataset Sentiment140  
- **Tahap 2.3:** Load dataset FordA    

# **Tahap 3 - Preprocessing Dataset**
Melakukan persiapan data sesuai kebutuhan tipe dataset.
- **Tahap 3.1:** Preprocessing BBC News — pembersihan teks, tokenisasi, encoding label, dan pembagian data train-test.  
- **Tahap 3.2:** Preprocessing Sentiment140 — pembersihan tweet (hapus URL, tanda baca, stopwords), tokenisasi, encoding label, pembagian data train-test.  
- **Tahap 3.3:** Preprocessing FordA — normalisasi data time series, pembagian data train-test, dan reshaping data agar sesuai input Transformer.


# **Tahap 4 - Definisi dan Inisialisasi Model Transformer**
Mendefinisikan arsitektur Transformer beserta layer-layer penting dan parameter yang akan dieksperimenkan.  

# **Tahap 5 – Training dan Evaluasi Model**
Melatih model pada ketiga dataset secara bergantian dengan parameter awal, serta evaluasi performa menggunakan metrik yang relevan.

- **Tahap 5.1:** Training & evaluasi pada BBC News  
- **Tahap 5.2:** Training & evaluasi pada Sentiment140  
- **Tahap 5.3:** Training & evaluasi pada FordA  

# **Tahap 6 – Analisis Hasil dan Visualisasi**
Membandingkan hasil performa model pada ketiga dataset, menampilkan tabel dan grafik hasil eksperimen, serta menganalisis faktor yang mempengaruhi performa.  

# **Tahap 7 – Kesimpulan dan Catatan**  
Menyimpulkan temuan eksperimen, mengidentifikasi parameter terbaik, dan membuat catatan penting untuk diskusi kelompok.